### Con un dataset de géneros se quiere comprobar si el contenido de las letras tiene que ver con el género al que pertenece la canción

The aim of this notebook is to make use of the word2vec model to find similar songs

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

Though non english artists were removed, the dataset contained Hindi lyrics of Lata Mangeshkar written in English. Therefore, I decided to remove all songs sung by her.

In [9]:
songs = pd.read_csv("C:/Users/Marino/Documents/UNIVERSIDAD/MasterBD/AnalisisDatosNoEstructurados/Practica/4_TEXT/data/lyrics.csv", header=0)
songs.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [10]:
songs.count()

index     362237
song      362235
year      362237
artist    362237
genre     362237
lyrics    266557
dtype: int64

In [12]:
songs.genre.unique()

array(['Pop', 'Hip-Hop', 'Rock', 'Metal', 'Country', 'Jazz', 'Electronic',
       'Folk', 'R&B', 'Indie'], dtype=object)

Al tratarse de una tabla con tantos registros, el ordenador no es capaz de procesar la función songvector, por lo que reducimos los registros de entrada a 100000 habiendo eliminado previamente aquelos registros de los géneros  'Not Available' y 'Other'

In [11]:
songs = songs.loc[songs["genre"]!="Not Available"]
songs = songs.loc[songs["genre"]!="Other"]

In [31]:
songs = songs[:100000]
songs.count()

index     100000
song      100000
year      100000
artist    100000
genre     100000
lyrics    100000
dtype: int64

To train the word2vec model, we first need to build its vocabulary. To do that, I iterated over each song and added it to an array that can later be fed to the model.

In [32]:
songs["lyrics"] = songs["lyrics"].replace("[^\w+]", " ", regex = True) 
songs.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,Oh baby how you doing You know I m gonna cut...
1,1,then-tell-me,2009,beyonce-knowles,Pop,playin everything so easy it s like you seem...
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn t hard to ...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,Oh oh oh I oh oh oh I Verse 1 If I wrote a...
4,4,black-culture,2009,beyonce-knowles,Pop,Party the people the people the party it s po...


In [33]:
songs.dtypes

index      int64
song      object
year       int64
artist    object
genre     object
lyrics    object
dtype: object

In [48]:
songs["lyrics"] = songs["lyrics"].astype(str)

In [49]:
songs.dtypes

index      int64
song      object
year       int64
artist    object
genre     object
lyrics    object
dtype: object

In [36]:
text_corpus = []
for song in songs['lyrics']:
    words = song.lower().split()
    text_corpus.append(words)


# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

100000


In [37]:
text_corpus[0]

['oh',
 'baby',
 'how',
 'you',
 'doing',
 'you',
 'know',
 'i',
 'm',
 'gonna',
 'cut',
 'right',
 'to',
 'the',
 'chase',
 'some',
 'women',
 'were',
 'made',
 'but',
 'me',
 'myself',
 'i',
 'like',
 'to',
 'think',
 'that',
 'i',
 'was',
 'created',
 'for',
 'a',
 'special',
 'purpose',
 'you',
 'know',
 'what',
 's',
 'more',
 'special',
 'than',
 'you',
 'you',
 'feel',
 'me',
 'it',
 's',
 'on',
 'baby',
 'let',
 's',
 'get',
 'lost',
 'you',
 'don',
 't',
 'need',
 'to',
 'call',
 'into',
 'work',
 'cause',
 'you',
 're',
 'the',
 'boss',
 'for',
 'real',
 'want',
 'you',
 'to',
 'show',
 'me',
 'how',
 'you',
 'feel',
 'i',
 'consider',
 'myself',
 'lucky',
 'that',
 's',
 'a',
 'big',
 'deal',
 'why',
 'well',
 'you',
 'got',
 'the',
 'key',
 'to',
 'my',
 'heart',
 'but',
 'you',
 'ain',
 't',
 'gonna',
 'need',
 'it',
 'i',
 'd',
 'rather',
 'you',
 'open',
 'up',
 'my',
 'body',
 'and',
 'show',
 'me',
 'secrets',
 'you',
 'didn',
 't',
 'know',
 'was',
 'inside',
 'no',
 

In [38]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 169.90734124183655 seconds ---


In [39]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [40]:
songs2vec.wv.most_similar("love")

[('unconditionally', 0.904120147228241),
 ('loving', 0.8920494318008423),
 ('unconditional', 0.8865437507629395),
 ('madly', 0.8854610323905945),
 ('vienna', 0.883482813835144),
 ('satisfies', 0.883428692817688),
 ('yearnin', 0.8826565742492676),
 ('darlin', 0.8789442181587219),
 ('gratitude', 0.8780825138092041),
 ('shemin', 0.8758261203765869)]

Words out of context

In [41]:
songs2vec.wv.doesnt_match("happy love hate table".split())

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'table'

In [42]:
songs2vec.most_similar(positive=['woman', 'king'], negative=['man'])
#queen

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('prince', 0.8462038636207581),
 ('queen', 0.8351825475692749),
 ('gypsy', 0.8193237781524658),
 ('kazoo', 0.8023850917816162),
 ('host', 0.8021976351737976),
 ('shepherd', 0.7987449169158936),
 ('lamb', 0.7978236675262451),
 ('peasant', 0.7932218313217163),
 ('israel', 0.7920053005218506),
 ('dilly', 0.7919646501541138)]

Con este corpus sí que aparecen los resultados esperados

Semantic distance between words

In [43]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [44]:
nearest_similarity_cosmul("paris", "france", "spain")
nearest_similarity_cosmul("paris", "france", "madrid")

paris es a france, lo que oklahoma es a spain
paris es a france, lo que daã es a madrid


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [45]:
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum

import sys
import time
start_time = time.time()

songs['song_vector'] = songs['lyrics'].apply(songVector)




C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

...

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem

AttributeError: 'int' object has no attribute 'reshape'

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [28]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

KeyError: 'song_vector'

In [ ]:
songs.size

In [ ]:
len(song_vectors)

I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [ ]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [ ]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

In [ ]:
x = two_dimensional_songs.X
x

**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['cancion'],
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['anyo'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)
#Grafico1

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()
#Grafico2

### LOOK FOR COMMON SONGS AND ANALYZE THE TEXT

In [ ]:
import plotly.express as px

fig = px.scatter_3d(two_dimensional_songs,x='X', y='Y', z='visitas',
              color='anyo')
fig.show()
#Grafico3

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['cancion'],
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['anyo'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)
#Grafico4

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()
#Grafico5

In [ ]:
import plotly.express as px

fig = px.scatter_3d(two_dimensional_songs,x='X', y='Y', z='visitas',
              color='anyo')
fig.show()
#Grafico6